### Define functions for sensor streaming

In [1]:
# Define SerialPortUnavailable exception

from serial.tools import list_ports

# Create Exception for serial port unavailable (explained at the end of the section)
class SerialPortUnavailable():

    def __init__(self):
        print("Chosen port is not available. Choose one of the following ports:")
        for port in list_ports.comports():
            print(f"    * {port.name}")

In [2]:
# Create streaming function

from datetime import datetime
import biosppy as bp
import serial
import re

def stream(port, file_path=None, cutoff_freq=4, sampling_rate=10):

    # Compute digital (FIR or IIR) filter coefficients with the given parameters
    coeffs_b, coeffs_a = bp.signals.tools.get_filter(ftype="FIR", band="lowpass", order=8, frequency=cutoff_freq, sampling_rate=sampling_rate,)
    # Create online filter object
    online_filter = bp.signals.tools.OnlineFilter(coeffs_b, coeffs_a)

    # Open new file every time to save raw and filtered data
    if file_path is None:
        now = datetime.now()
        file_path = f'./sample_{now.strftime("%d-%m-%Y_%H-%M-%S")}.txt'
    f = open(file_path, 'a')

    pattern = re.compile(r'\d+,\d+,\d\n') # pattern to make sure the message received from the serial port is in the format "{value},{value}\n"
    message = ""

    try:
        with serial.Serial(port, baudrate=9600, timeout=0) as arduino: # initiate serial communication
            
            try:

                while True:
                    arduino_bytes = arduino.readline() # read message from arduino and decode it from bytes to string
                    
                    if arduino_bytes != message:
                        message = message + arduino_bytes.decode() # in case the full line comes in different messages, concatenate it

                    match = pattern.search(message) # make sure the message received from the serial port is in the format "{value},{value}\n"
                    if match is not None: 
                    
                        print(f"complete message: {match.group().strip()}")
                        message = ""

                        try:
                            matched_message = match.group().strip()
                            parsed_time = matched_message.split(",")[0] # parse timestamp
                            parsed_data = [int(matched_message.split(",")[1])] # parse signal data point
                            parsed_annot = matched_message.split(",")[2] # parse pushbuton annotations
                            filtered_data = online_filter.filter(parsed_data)["filtered"] # pass the signal data point to the online filter
                            f.write(f"{parsed_time},{parsed_data[0]},{filtered_data[0]},{parsed_annot}\n") # write all to file

                        except Exception as e:
                            print(f"error: {e} | message: {matched_message}")
                            continue

            except KeyboardInterrupt:
                print("\nAcquisition interrupted by you")

            except Exception as e:
                print(e)

            finally: # regardless of the exception that was raised, the following code will run
                arduino.close()
                f.close()
                print('\nSerial communication and file closed')

    except serial.SerialException:
        SerialPortUnavailable()
        f.close()
        import os
        os.remove(file_path)



### Define functions for visualization

In [3]:
# Define function that deals with annotations

def get_annot_data(data, annotation_labels, sampling_rate=10): 

    # Get pushbutton annotations and remove duplicates
    annots = data.index[data['annot'] == True].tolist()

    sampling_period = 1000/sampling_rate # in milliseconds
    last_annot = -(sampling_period + 1)
    annots_new = []

    for annot in annots:
        if annot != last_annot + sampling_period:
            annots_new += [annot]
        last_annot = annot

    if (len(annots_new) % 2) != 0: annots_new.pop()

    if len(annots_new)/2 > len(annotation_labels):
        print(f"Not enough annotation labels (expected {int(len(annots_new)/2)}, got {len(annotation_labels)})")
    elif len(annots_new)/2 < len(annotation_labels):
        print(f"Not enough annotations (expected {len(annotation_labels)}, got {int(len(annots_new)/2)})")

    if len(annots_new)/2 >= len(annotation_labels):
        for i in range(1,len(annotation_labels)+1):
            data.loc[annots_new[2*i-2]:annots_new[2*i-1], "annot-label"] = annotation_labels[i-1]
    else:
        for ii,i in enumerate(range(0,len(annots_new),2)):
            data.loc[annots_new[i]:annots_new[i+1], "annot-label"] = annotation_labels[ii]

    return annots_new, data

In [4]:
# Create function to plot raw acquisition

import plotly.express as px
import pandas as pd

color_palette=['#C98986', '#8B575C', '#7899D4', '#9EC1A3', '#1F363D'] 

def plot_data(data, remove_nan=False, to_plot=["raw", "filtered"]):

    if remove_nan:
        data.loc[data['annot-label'].isna(), to_plot] = None
    
    fig = px.line(data, y=to_plot, labels = {"index":"timestamps (ms)", "value":"raw"}, color_discrete_sequence=color_palette)
    
    try:
        annots = data['annot-label'].unique()

        for a in annots:
            if not pd.isna(a):
                idx = data.index[data['annot-label'] == a].tolist()
                fig.add_vrect(x0=idx[0], x1=idx[-1], 
                                annotation_text=a, annotation_position="top left",
                                fillcolor="black", opacity=0.2, line_width=0)
    except:
        pass
                            
    fig.show()

In [5]:
# Create function to make violin plot

from plotly.subplots import make_subplots
import plotly.graph_objects as go

color_palette=['#C98986', '#8B575C', '#7899D4', '#9EC1A3', '#1F363D'] 

def plot_box(data, to_plot="filtered", scaled=True):

    data = data[data['annot-label'].notna()]
    #fig = px.violin(data, y=to_plot, color="annot-label", color_discrete_sequence=color_palette)
    
    annots = data['annot-label'].unique()
    amplitudes = []

    if scaled:
        for a in annots:
            data_a = data[data['annot-label'] == a][to_plot]
            amplitudes += [(data_a.max() - data_a.min()).max()]
        range = max(amplitudes)

        fig = make_subplots(rows=1, cols=len(annots))

        for i,a in enumerate(annots):
            data_to_plot = data[data["annot-label"]==a][to_plot]
            fig.add_trace(go.Box(y=data_to_plot), row=1, col=(i+1))
            add_to_range = (range - (data_to_plot.max()-data_to_plot.min()))/2
            fig.update_yaxes(range=[data_to_plot.min()-add_to_range, data_to_plot.max()+add_to_range], row=1, col=(i+1))
            fig.update_yaxes(range=[data[to_plot].min(), data[to_plot].max()], row=1, col=(i+1))
            fig.update_traces(name=a, row=1, col=(i+1))
    else: 
        fig = px.violin(data, y=to_plot, color="annot-label", facet_col="annot-label")
                            
    fig.show()

### Data Streaming - Arduino

In [ ]:
sampling_rate = 10 #Hz

now = datetime.now()
file_path = f'./sample_{now.strftime("%d-%m-%Y_%H-%M-%S")}.txt'

stream(port='/dev/cu.usbmodem101', file_path=file_path, cutoff_freq=4, sampling_rate=sampling_rate)

In [ ]:
#file_path = "./distance_25-11-2022_13-39-33.txt"

data = pd.read_csv(file_path, names=["raw", "filtered","annot"], index_col=0) # in this case, the time column is set as the index

plot_data(data, remove_nan=False, to_plot=["raw", "filtered"])

### Data Streaming - Sense

In [10]:
from sense import main
from datetime import datetime

now = datetime.now()

address = "/dev/cu.ScientISST-54-96"
channels = "1"
fs = 10
file_path = f'./sense_{now.strftime("%d-%m-%Y_%H-%M-%S")}.txt'
main(address, fs, channels, file_path)



Connecting to /dev/cu.ScientISST-54-96...
ScientISST version: 1.0
ScientISST Board Vref: 1114
ScientISST Board ADC Attenuation Mode: 0
Connected!
Start acquisition
Saving data to ./sense_09-01-2023_14-08-58.txt
NSeq	I1	I2	O1	O2	AI1_raw	AI1_mv


Error checking CRC4Error checking CRC4Error checking CRC4Error checking CRC4Error checking CRC4Error checking CRC4Error checking CRC4Error checking CRC4Error checking CRC4Error checking CRC4Error checking CRC4Error checking CRC4Error checking CRC4Error checking CRC4Error checking CRC4Error checking CRC4Error checking CRC4Error checking CRC4Error checking CRC4Error checking CRC4Error checking CRC4Error checking CRC4Error checking CRC4Error checking CRC4Error checking CRC4Error checking CRC4Error checking CRC4Error checking CRC4Error checking CRC4Error checking CRC4Error checking CRC4Error checking CRC4Error checking CRC4Error checking CRC4Error checking CRC4Error checking CRC4Error checking CRC4Error checking CRC4Error checking CRC4Error checking CRC4Error checking CRC4Error checking CRC4Error checking CRC4Error checking CRC4Error checking CRC4Error checking CRC4Error checking CRC4Error checking CRC4Error checking CRC4Error checking CRC4Error checking CRC4Error checking CRC4Error checki

Stop acquisition
Disconnected


In [9]:
import pandas as pd
import biosppy as bp

chn = "1"

data = pd.read_csv(file_path, skiprows=1, header=0, delimiter="\t", skipfooter=2, engine='python')
data[f"AI{chn}_filtered"], _, _ = bp.signals.tools.filter_signal(signal=data[f"AI{chn}_raw"].to_numpy(), ftype='FIR', band='lowpass', order=8, frequency=1, sampling_rate=10.0)

px.line(data, y=[f"AI{chn}_raw", f"AI{chn}_filtered"], color_discrete_sequence=color_palette)

ValueError: The length of the input vector x must be greater than padlen, which is 27.

### Analize distance between sensor and magnet (angle = 0º)

##### Stream sensor data

In [ ]:
sampling_rate = 10 #Hz

now = datetime.now()
file_path = f'./distance_{now.strftime("%d-%m-%Y_%H-%M-%S")}.txt'

stream(port='/dev/cu.usbmodem101', file_path=file_path, cutoff_freq=4, sampling_rate=sampling_rate)

##### Plot data

In [ ]:
#file_path = "./distance_11-11-2022_15-38-35.txt"
annotation_labels = [f"{d}mm" for d in range(0, 20, 2)] + ["21mm"]

data = pd.read_csv(file_path, names=["raw", "filtered","annot"], index_col=0) # in this case, the time column is set as the index
annots_new, data = get_annot_data(data, annotation_labels=annotation_labels)

plot_data(data, remove_nan=False, to_plot=["raw", "filtered"])

##### Make box-plot

In [ ]:
plot_box(data, to_plot="filtered", scaled=True)

### Analize angle between sensor and magnet (distance = 0mm)

##### Stream sensor data

In [ ]:
sampling_rate = 10 #Hz

now = datetime.now()
file_path = f'./angle_{now.strftime("%d-%m-%Y_%H-%M-%S")}.txt'

stream(port='/dev/tty.usbserial-0001', file_path=file_path, cutoff_freq=4, sampling_rate=sampling_rate)

##### Plot data

In [ ]:
file_path = "./angle_11-11-2022_15-52-30.txt"
annotation_labels = [f"{a}º" for a in range(0, 95, 10)]

data = pd.read_csv(file_path, names=["raw", "filtered","annot"], index_col=0) # in this case, the time column is set as the index
annots_new, data = get_annot_data(data, annotation_labels=annotation_labels)

plot_data(data, remove_nan=True, to_plot=["raw", "filtered"])

##### Make box-plot

In [ ]:
plot_box(data, to_plot="filtered", scaled=True)